## Inference by direct Request

In [11]:
import requests
import json

class GeminiClient:
    def __init__(self, api_key_file):
        with open(api_key_file, 'r') as file:
            self.api_key = file.read().strip()
        self.url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent"

    def call_api(self, user_input):
        """Send a request to the Gemini API and return the response data."""
        headers = {"Content-Type": "application/json"}
        data = {
            "contents": [{
                "parts": [{"text": user_input}]
            }]
        }
        response = requests.post(
            f"{self.url}?key={self.api_key}",
            headers=headers,
            json=data
        )
        return response.json()

    def process_input(self, user_input):
        """Process the user input through the Gemini API and save response to JSON."""
        response_data = self.call_api(user_input)
        print("Full response:", response_data)
        self.save_response(response_data)
        return response_data.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text', '')

    def save_response(self, data):
        """Save the API response data to a JSON file."""
        with open('request.json', 'w') as f:
            json.dump(data, f, indent=4)

# Usage
api_key_file = 'gemini_api.txt'
client = GeminiClient(api_key_file)
output = client.process_input("I am having a stomach and slight headache, what should I do?")
print("Output Text:", output)

Full response: {'candidates': [{'content': {'parts': [{'text': "I'm sorry to hear you're not feeling well! I'm an AI and can't provide medical advice. It's important to seek help from a healthcare professional for any health concerns. \n\nHere are some things you can try that might help you feel better in the meantime:\n\n* **Stay hydrated:** Drink plenty of water or clear broth.\n* **Eat bland foods:** Avoid greasy, spicy, or acidic foods. Toast, crackers, and rice are good options.\n* **Rest:** Get plenty of sleep and avoid strenuous activities.\n* **Over-the-counter medication:** You may consider taking an over-the-counter pain reliever like ibuprofen or acetaminophen. \n\nIf your symptoms worsen or persist, or if you experience any of the following, please seek medical attention immediately:\n\n* Fever\n* Vomiting or diarrhea\n* Severe abdominal pain\n* Blood in your stool\n* Difficulty breathing\n\n**Remember, it's crucial to consult a doctor for a proper diagnosis and treatment.*

## Inference by Langchain

In [17]:
# !pip install langchain_google_genai

import os
import json
import markdown
from langchain_google_genai import ChatGoogleGenerativeAI

def load_api_key(file_path):
    """Load the API key from a specified file."""
    with open(file_path, 'r') as file:
        return file.read().strip()

# Load API
api_key = load_api_key('gemini_api.txt')

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=api_key)
result = llm.invoke("I am having a stomach and slight headache, what should I do?")
print(result)

# Save the response
data_to_save = {
    "response": result.content
}

with open('langchain.json', 'w') as json_file:
    json.dump(data_to_save, json_file, indent=4)

content="I understand you're not feeling well. I am an AI and cannot provide medical advice. It's important to consult a medical professional for any health concerns. \n\nHere are some general tips that might help while you wait to see a doctor:\n\n* **Hydrate:** Drink plenty of fluids, especially water, to stay hydrated.\n* **Rest:** Give your body time to rest and recover.\n* **Eat bland foods:** Avoid greasy, spicy, or acidic foods that might irritate your stomach. Stick to bland foods like toast, crackers, or rice.\n* **Over-the-counter medication:** Consider taking over-the-counter medications like Pepto-Bismol for stomach upset or Tylenol for headache, but always follow the instructions on the label.\n\n**However, if your symptoms are severe or persistent, seek medical attention immediately. This could include:**\n\n* **Severe pain or vomiting**\n* **Fever**\n* **Blood in your vomit or stool**\n* **Difficulty swallowing**\n* **Sudden weight loss**\n\nRemember, it's always best to

## Test RAG for Gemini

In [19]:
# !pip install --quiet --upgrade langchain langchain-community langchain-chroma

import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

def load_api_key(file_path):
    """Load the API key from a specified file."""
    with open(file_path, 'r') as file:
        return file.read().strip()

api_key = load_api_key('gemini_api.txt')

os.environ[api_key] = getpass.getpass()

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=api_key)

In [ ]:
# TODO write code from here https://www.youtube.com/watch?app=desktop&v=8xVmzoP1lks&ab_channel=KarndeepSingh